In [1]:
import time
from time import sleep
import re
from random import randint
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook
import pandas as pd

In [2]:
errorList = []

def crawlStart(year):
    url = 'https://movie.daum.net/boxoffice/yearly?year='+str(year)
    browser = webdriver.Chrome()
    # 브라우저 실행
    browser.get(url)
    
    movieName, netScore, criScore, totalAudiance = movieInfo(browser)
    
    # 데이터프레임 만들기 (dict형식)
    df = pd.DataFrame(
        {"title": movieName,
        "netizenScore": netScore,
        "criticalScore": criScore,
        "totalAudiance": totalAudiance})
    
    return df

In [3]:
def movieInfo(browser):
    # 제목 추출
    movieList = browser.find_elements_by_class_name("name_movie")
    movieName = [movie.text for movie in movieList]

    # 평점 리스트
    scoreList = browser.find_elements_by_class_name("info_grade")
    scoreList = [score.text.split() for score in scoreList]

    netScore = []
    criScore = []
    for score in scoreList:
        # 네티즌 평점
        netScore.append(float(score[2]) + float(score[4])/10)
        # 기자/평론가 평점
        try:
            criScore.append(float(score[7]) + float(score[9])/10)
        except Exception as e :
            criScore.append(None)
            errorList.append(e)
    
    totalAudiance = []
    # movieID로 가져와서 movieList변경
    movieList = [movie.get_attribute("href").split("=")[1] for movie in movieList]

    for movieID in tqdm_notebook(movieList):
        try:
            browser.get("https://movie.daum.net/moviedb/main?movieId=" + movieID)
            totalAudiance.append(float("".join(browser.find_element_by_id("totalAudience").text[:-1].split(","))))
        except:
            totalAudiance.append(None)
        sleep(1)
    return movieName, netScore, criScore, totalAudiance

In [11]:
years = [2018, 2019]
for year in years:
    data = pd.concat([data, crawlStart(year)])

In [13]:
data.to_csv("movieInfo.csv")